In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, LeakyReLU, ReLU
import optuna
from tensorflow import keras

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

# Load the datasets
dataset1 = pd.read_csv('/content/drive/MyDrive/tata/2018.csv')
dataset2 = pd.read_csv('/content/drive/MyDrive/tata/2019.csv')
dataset3 = pd.read_csv('/content/drive/MyDrive/tata/2020.csv')


In [5]:
dataset1

,DateTime,RH,Temp,Load
0,01-01-2018 00:00,80.423000,18.160,174.542804
1,01-01-2018 00:15,81.471000,18.064,175.981604
2,01-01-2018 00:30,82.675000,17.927,178.636804
3,01-01-2018 00:45,84.552000,17.745,180.045604
4,01-01-2018 01:00,86.663000,17.751,182.254000
...,...,...,...,...
35035,31-12-2018 22:45,63.469773,17.647,181.566544
35036,31-12-2018 23:00,61.612709,17.761,181.542448
35037,31-12-2018 23:15,61.666345,17.689,181.600600
35038,31-12-2018 23:30,60.732709,17.818,183.150424


In [6]:
df_all = pd.concat((dataset1, dataset2, dataset3), axis=0)


In [7]:
df_all = df_all.set_index('DateTime')

In [8]:

df_rh = df_all.drop(['Load', 'Temp'], axis=1)

In [9]:
df_rh.head()

,RH
DateTime,
01-01-2018 00:00,80.423
01-01-2018 00:15,81.471
01-01-2018 00:30,82.675
01-01-2018 00:45,84.552
01-01-2018 01:00,86.663


In [10]:
np_rh = df_rh.to_numpy()
np_rh

array([[80.423],
       [81.471],
       [82.675],
       ...,
       [82.6  ],
       [82.37 ],
       [82.5  ]])

In [11]:
np_rh = np_rh.astype(np.float32)
np.save('/content/drive/MyDrive/tata/combined_RH_data.npy', np_rh)

In [12]:
np_rh

array([[80.423],
       [81.471],
       [82.675],
       ...,
       [82.6  ],
       [82.37 ],
       [82.5  ]], dtype=float32)

In [13]:
Data = np.load('/content/drive/MyDrive/tata/combined_RH_data.npy',allow_pickle=True)

In [14]:
Data

array([[80.423],
       [81.471],
       [82.675],
       ...,
       [82.6  ],
       [82.37 ],
       [82.5  ]], dtype=float32)

In [15]:
# scaler = MinMaxScaler()
# model=scaler.fit(Data.reshape(Data.shape[0],Data.shape[1]))
# scaled_data=model.transform(Data.reshape(Data.shape[0],Data.shape[1]))
# X = []
# y = []
# for i in range(14 ,scaled_data.shape[0]-7):
#    X.append(scaled_data[i-14:i])
#    y.append(scaled_data[i:i+7])
# X = np.array(X)
# y = np.array(y)
# y=y.reshape(X.shape[0],y.shape[1]*y.shape[2])
# X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [16]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
humidity_scaled = scaler.fit_transform(Data)

# Prepare the dataset for DNN
n_steps = 192  # Number of time steps per sample
X, y = [], []
for i in range(672, len(humidity_scaled) - n_steps):
    X.append(humidity_scaled[i-672:i])
    y.append(humidity_scaled[i:i+192])
X, y = np.array(X), np.array(y)

y=y.reshape(X.shape[0],y.shape[1]*y.shape[2])
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [19]:
def create_model(trial):

  # Create the LSTM model
  model = keras.Sequential([
      keras.layers.Input(shape=(X_train.shape[1],)),
      keras.layers.Dense(trial.suggest_int('lstm_1_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_2_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_3_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_4_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_5_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_6_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_7_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_8_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_9_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_10_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_11_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(trial.suggest_int('lstm_12_units',y_train.shape[1], X_train.shape[1]), activation='ReLU'),
      keras.layers.Dense(y_train.shape[1])
  ])

  # Compile the model
  optimizer_name = trial.suggest_categorical("optimizer", ["adam", "SGD", "RMSprop", "Adadelta"])
  # optimizer = tf.keras.optimizers.Adam(learning_rate=trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True))
  model.compile(optimizer=optimizer_name, loss='mse', metrics=['mse'])

  return model

def objective(trial):
    model = create_model(trial)
    epochs = trial.suggest_int("epochs", 4, 30,step=4, log=False)
    batchsize = trial.suggest_int("batchsize", 4, 30,step=4, log=False)

    # Train the model
    model.fit(X_train, y_train, batch_size=batchsize, epochs=epochs, validation_split = 0.1,verbose=1)

    # Evaluate the model
    scores = model.evaluate(X_test, y_test, verbose=0)
    return scores[1]

study = optuna.create_study()
study.optimize(objective, n_trials=4)

print(study.best_params)

[I 2023-07-18 12:21:41,221] A new study created in memory with name: no-name-40dc75b2-83e0-4945-904c-e7271d0c9383


Epoch 1/16
4403/4403 [==============================] - 32s 6ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.0207 - val_mse: 0.0207
Epoch 2/16
4403/4403 [==============================] - 27s 6ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.0110 - val_mse: 0.0110
Epoch 3/16
4403/4403 [==============================] - 26s 6ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.0104 - val_mse: 0.0104
Epoch 4/16
4403/4403 [==============================] - 27s 6ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.0105 - val_mse: 0.0105
Epoch 5/16
4403/4403 [==============================] - 26s 6ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0114 - val_mse: 0.0114
Epoch 6/16
4403/4403 [==============================] - 26s 6ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0104 - val_mse: 0.0104
Epoch 7/16
4403/4403 [==============================] - 27s 6ms/step - loss: 0.0056 - mse: 0.0056 - val_loss: 0.0113 - val_mse: 0.0113
Epoch 8/16
4403/4403 [==============================] -

[I 2023-07-18 12:29:08,055] Trial 0 finished with value: 0.00967805739492178 and parameters: {'lstm_1_units': 466, 'lstm_2_units': 407, 'lstm_3_units': 284, 'lstm_4_units': 665, 'lstm_5_units': 488, 'lstm_6_units': 618, 'lstm_7_units': 627, 'lstm_8_units': 523, 'lstm_9_units': 385, 'lstm_10_units': 398, 'lstm_11_units': 581, 'lstm_12_units': 582, 'optimizer': 'adam', 'epochs': 16, 'batchsize': 16}. Best is trial 0 with value: 0.00967805739492178.


Epoch 1/28
2516/2516 [==============================] - 22s 6ms/step - loss: 0.0147 - mse: 0.0147 - val_loss: 0.0165 - val_mse: 0.0165
Epoch 2/28
2516/2516 [==============================] - 15s 6ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.0108 - val_mse: 0.0108
Epoch 3/28
2516/2516 [==============================] - 15s 6ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.0108 - val_mse: 0.0108
Epoch 4/28
2516/2516 [==============================] - 16s 6ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.0109 - val_mse: 0.0109
Epoch 5/28
2516/2516 [==============================] - 15s 6ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0105 - val_mse: 0.0105
Epoch 6/28
2516/2516 [==============================] - 15s 6ms/step - loss: 0.0058 - mse: 0.0058 - val_loss: 0.0103 - val_mse: 0.0103
Epoch 7/28
2516/2516 [==============================] - 16s 6ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0109 - val_mse: 0.0109
Epoch 8/28
2516/2516 [==============================] -

[I 2023-07-18 12:36:31,089] Trial 1 finished with value: 0.0091515788808465 and parameters: {'lstm_1_units': 419, 'lstm_2_units': 405, 'lstm_3_units': 366, 'lstm_4_units': 256, 'lstm_5_units': 541, 'lstm_6_units': 313, 'lstm_7_units': 241, 'lstm_8_units': 305, 'lstm_9_units': 484, 'lstm_10_units': 607, 'lstm_11_units': 415, 'lstm_12_units': 595, 'optimizer': 'adam', 'epochs': 28, 'batchsize': 28}. Best is trial 1 with value: 0.0091515788808465.


Epoch 1/4
3522/3522 [==============================] - 28s 6ms/step - loss: 0.0144 - mse: 0.0144 - val_loss: 0.0218 - val_mse: 0.0218
Epoch 2/4
3522/3522 [==============================] - 22s 6ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.0215 - val_mse: 0.0215
Epoch 3/4
3522/3522 [==============================] - 21s 6ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.0210 - val_mse: 0.0210
Epoch 4/4
3522/3522 [==============================] - 22s 6ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.0217 - val_mse: 0.0217


[I 2023-07-18 12:38:07,637] Trial 2 finished with value: 0.01353403925895691 and parameters: {'lstm_1_units': 440, 'lstm_2_units': 455, 'lstm_3_units': 431, 'lstm_4_units': 661, 'lstm_5_units': 487, 'lstm_6_units': 657, 'lstm_7_units': 587, 'lstm_8_units': 557, 'lstm_9_units': 661, 'lstm_10_units': 462, 'lstm_11_units': 544, 'lstm_12_units': 643, 'optimizer': 'adam', 'epochs': 4, 'batchsize': 20}. Best is trial 1 with value: 0.0091515788808465.


Epoch 1/12
8805/8805 [==============================] - 59s 6ms/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.0218 - val_mse: 0.0218
Epoch 2/12
8805/8805 [==============================] - 52s 6ms/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.0213 - val_mse: 0.0213
Epoch 3/12
8805/8805 [==============================] - 52s 6ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.0210 - val_mse: 0.0210
Epoch 4/12
8805/8805 [==============================] - 53s 6ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.0238 - val_mse: 0.0238
Epoch 5/12
8805/8805 [==============================] - 54s 6ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.0219 - val_mse: 0.0219
Epoch 6/12
8805/8805 [==============================] - 56s 6ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.0218 - val_mse: 0.0218
Epoch 7/12
8805/8805 [==============================] - 52s 6ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.0215 - val_mse: 0.0215
Epoch 8/12
8805/8805 [==============================] -

[I 2023-07-18 12:49:33,976] Trial 3 finished with value: 0.014155068434774876 and parameters: {'lstm_1_units': 521, 'lstm_2_units': 581, 'lstm_3_units': 639, 'lstm_4_units': 366, 'lstm_5_units': 206, 'lstm_6_units': 513, 'lstm_7_units': 633, 'lstm_8_units': 238, 'lstm_9_units': 223, 'lstm_10_units': 217, 'lstm_11_units': 495, 'lstm_12_units': 667, 'optimizer': 'adam', 'epochs': 12, 'batchsize': 8}. Best is trial 1 with value: 0.0091515788808465.


{'lstm_1_units': 419, 'lstm_2_units': 405, 'lstm_3_units': 366, 'lstm_4_units': 256, 'lstm_5_units': 541, 'lstm_6_units': 313, 'lstm_7_units': 241, 'lstm_8_units': 305, 'lstm_9_units': 484, 'lstm_10_units': 607, 'lstm_11_units': 415, 'lstm_12_units': 595, 'optimizer': 'adam', 'epochs': 28, 'batchsize': 28}


In [20]:
# Define the architecture of the DNN model
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(222, activation='ReLU'),
    keras.layers.Dense(232, activation='ReLU'),
    keras.layers.Dense(283, activation='ReLU'),
    keras.layers.Dense(180, activation='ReLU'),
    keras.layers.Dense(248, activation='ReLU'),
    keras.layers.Dense(157, activation='ReLU'),
    keras.layers.Dense(483, activation='ReLU'),
    keras.layers.Dense(257, activation='ReLU'),
    keras.layers.Dense(83, activation='ReLU'),
    keras.layers.Dense(y_train.shape[1])
])

# Compile the DNN model
model.compile(optimizer='adam', loss='mse')

# Train the DNN model
history = model.fit(X_train, y_train, epochs=30, batch_size=30, validation_split=0.1,verbose=1)


model.summary()

Epoch 1/30
2348/2348 [==============================] - 16s 5ms/step - loss: 0.0107 - val_loss: 0.0106
Epoch 2/30
2348/2348 [==============================] - 12s 5ms/step - loss: 0.0063 - val_loss: 0.0104
Epoch 3/30
2348/2348 [==============================] - 12s 5ms/step - loss: 0.0060 - val_loss: 0.0102
Epoch 4/30
2348/2348 [==============================] - 12s 5ms/step - loss: 0.0059 - val_loss: 0.0099
Epoch 5/30
2348/2348 [==============================] - 12s 5ms/step - loss: 0.0057 - val_loss: 0.0110
Epoch 6/30
2348/2348 [==============================] - 12s 5ms/step - loss: 0.0056 - val_loss: 0.0102
Epoch 7/30
2348/2348 [==============================] - 12s 5ms/step - loss: 0.0055 - val_loss: 0.0105
Epoch 8/30
2348/2348 [==============================] - 11s 5ms/step - loss: 0.0054 - val_loss: 0.0108
Epoch 9/30
2348/2348 [==============================] - 12s 5ms/step - loss: 0.0053 - val_loss: 0.0101
Epoch 10/30
2348/2348 [==============================] - 12s 5ms/step - l

In [22]:
# Evaluate the performance of the DNN model
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error:", mse)

816/816 [==============================] - 2s 3ms/step - loss: 0.0090
Mean Squared Error: 0.009021883830428123
